In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time

# Init

In [ ]:
import fp_utils

In [ ]:
fp_utils.settings.init_fp_utils()

# Data io

In [ ]:
data_path = Path('../data')

In [ ]:
from fp_utils.data_io import DataFrameLoader

In [ ]:
df = DataFrameLoader.pickle(data_path / '1e5.pickle')
df.shape

In [ ]:
cols2 = np.load('../data/cols.npy')

In [ ]:
df2 = DataFrameLoader.pickle('../data/1e5.pickle', columns=cols2)

# Speed testing

In [ ]:
from fp_utils.finders import BFDriveFinder, BFRamFinder, BFPDriveFinder, BFPRamFinder
from fp_utils.tests import FinderSpeedTester
from fp_utils.packing_mixins import PickleMixin

In [ ]:
class BFD(BFDriveFinder, PickleMixin):
    pass

class BFPD(BFPDriveFinder, PickleMixin):
    pass

In [ ]:
bf_drive_finder = BFD(df, data_path)

In [ ]:
bf_ram_finder = BFRamFinder(df)

In [ ]:
bfp_drive_finder = BFPD(df, data_path)

In [ ]:
bfp_ram_finder = BFPRamFinder(df)

In [ ]:
finders = [bf_drive_finder, bfp_drive_finder, bf_ram_finder, bfp_ram_finder]

In [ ]:
speed_tester = FinderSpeedTester(finders)

In [ ]:
fp_utils.settings.set_progress_bar(False)
speed_tester.test_random(df, 2, verbose=True) 

In [ ]:
stat = speed_tester.test_random(df, 2, ans_count=10, verbose=True)

In [ ]:
stat.as_plot(figsize=(8, 5))

In [ ]:
stat.as_boxplot(figsize=(8, 5))

# Corectness testing

In [ ]:
from fp_utils.tests import FinderCorrectnessTester

In [ ]:
correctness_tester = FinderCorrectnessTester(finders[0], finders[1:])

In [ ]:
correctness_tester.test_random(df, 2)

In [ ]:
correctness_tester.test_all(df.sample(2), verbose=True)

In [ ]:
wa_finder = BFRamFinder(df.sample(int(len(df) * 0.5)))

In [ ]:
correctness_tester = FinderCorrectnessTester(bf_drive_finder, [bfp_drive_finder, wa_finder])

In [ ]:
correctness_tester.test_random(df, 2, verbose=True)

# Draw

In [ ]:
from fp_utils.draw import MoleculeDrawer

In [ ]:
fingerprint = pd.read_pickle("../data/f.pickle")
fingerprint.name = 'NCCc1ccc4c(c1)OCCO4'
fingerprint = fingerprint[df.columns]

In [ ]:
MoleculeDrawer.draw_one(fingerprint.name)

In [ ]:
answers = list(bf_ram_finder.find(fingerprint, ans_count=10))